In [1]:
import numpy as np
import pandas as pd
from datetime import timedelta
import yfinance as yf
import string

p = 14

path = ""

mmf = pd.read_excel(path + "Market Manipulation Cases\\Market Manipulation Cases.xlsx")

mmf['Interval'] = mmf['End of MM'].sub(mmf['Start of MM'], axis=0)
mmf['Interval'] = mmf['Interval'] + timedelta(days = 1)
mmf['SideInterval'] = mmf['Interval'] * 2 + timedelta(days = 3)
mmf['Start'] = mmf['Start of MM'] - mmf['SideInterval']
mmf['Start'] = mmf['Start'] - timedelta(days = p)
mmf['End'] = mmf['End of MM'] + mmf['SideInterval']

for i in mmf.index:
    if mmf.iloc[i, 3] == 1:
        data = yf.download(mmf.iloc[i, 0], mmf.iloc[i, 7], mmf.iloc[i, 8])
        data['Manipulated'] = [1 if x  >= mmf.iloc[i, 1] and x <= mmf.iloc[i, 2] else 0 for x in data.index]
    
        indeces_to_delete = data[ data['Volume'] == 0 ].index
  
        data.drop(indeces_to_delete, inplace = True)
        string = path + "Raw Separate Daily Data\\" + mmf.iloc[i, 0] + "_" + str(int(mmf.iloc[i, 4])) + ".xlsx"
        data.to_excel(string, index = False)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- SUSS: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- GRO: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[

In [2]:
#normalizaing and combining data

import math
np.seterr('raise')

n = int(p - 2*(p/7))

mmf = pd.read_excel(path + "Market Manipulation Cases\\Market Manipulation Cases.xlsx")

alldays= pd.DataFrame(columns = ['Open','High', 'Low', 'Close', 'Adj Close', 'Volume', 'Manipulated', 'Company'])

for ind in mmf.index:
    if mmf.iloc[ind, 3] == 1:
        i = mmf.iloc[ind, 0]
        appear = str(int(mmf.iloc[ind, 4]))
        comp = pd.read_excel(path + "Raw Separate Daily Data\\" + i + "_" + appear + ".xlsx")
        newcomp = pd.DataFrame(columns = ['Open','High', 'Low', 'Close', 'Adj Close', 'Volume', 'Manipulated'])
        for j in comp.index:
            if j > n - 1:
                d = j - n
                ap = (comp.iloc[d:j + 1, 0] + comp.iloc[d:j + 1, 3])/2
                v = comp.iloc[d:j + 1, 5]
                ap_by_v = ap*v
                ap_by_v_sum = ap_by_v.sum(axis=0)
                v = comp.iloc[d:j + 1, 5]
                v_sum = v.sum(axis=0)
                aap = ap_by_v_sum/v_sum
                av = v_sum/ (n + 1)

                ap_dif = ap - aap
                ap_dif_s = ap_dif**2
                ap_dif_s_by_v = ap_dif_s*v
                ap_dif_s_by_v_sum = ap_dif_s_by_v.sum(axis=0)
                aap_var = ap_dif_s_by_v_sum / v_sum

                v_dif = v - av
                v_dif_s = v_dif**2
                v_dif_s_sum = v_dif_s.sum(axis=0)
                v_var = v_dif_s_sum / (n)
                
                row = []
                for l in range(0, 5):
                    row.append((comp.iloc[j, l] - aap) / math.sqrt(aap_var))
                row.append((comp.iloc[j, 5] - av) / math.sqrt(v_var))
                row.append(int(comp.iloc[j, 6]))

                newcomp.loc[len(newcomp)] = row

        newcomp['Company'] = [i]*len(newcomp)
        alldays = pd.concat([alldays, newcomp])
    

string = path + "Normalized data\\" + "AllDays" + ".xlsx"
alldays.to_excel(string, index = False)

In [3]:
#splitting by the fact of manipulation

manip_days = pd.DataFrame(columns = alldays.columns)
not_manip_days = pd.DataFrame(columns = alldays.columns)

for i in alldays.index:
    if alldays.iloc[i, 6] == 1:
        manip_days.loc[len(manip_days)] = alldays.iloc[i]
    else:
        not_manip_days.loc[len(not_manip_days)] = alldays.iloc[i]

In [13]:
#randomizing the days
import random

def randomize_rows(my_dataframe):
    inds = my_dataframe.index
    inds = list(inds)
    rand_inds = random.sample(inds, len(inds))
    rand_days = pd.DataFrame(columns = my_dataframe.columns)
    for i in rand_inds:
        rand_days.loc[len(rand_days)] = my_dataframe.loc[i]
    return rand_days

manip_days = randomize_rows(manip_days)
not_manip_days = randomize_rows(not_manip_days)

manip_days.to_excel(path + "Normalized data\\" + "Manipulated Days" + ".xlsx", index = False)
not_manip_days.to_excel(path + "Normalized data\\" + "Not Manipulated Days" + ".xlsx", index = False)

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

manip_days = pd.read_excel(path + "Normalized data\\" + "Manipulated Days" + ".xlsx")
not_manip_days = pd.read_excel(path + "Normalized data\\" + "Not Manipulated Days" + ".xlsx")

x_m = manip_days.iloc[:, 0:6] 
y_m = manip_days.iloc[:, 6]

x_n = not_manip_days.iloc[:, 0:6] 
y_n = not_manip_days.iloc[:, 6]

l_m = int(len(x_m)*0.8)
l_n = int(len(x_n)*0.8)

x_train = pd.concat([x_m.iloc[:l_m, :], x_n.iloc[:l_n, :]])
x_test = pd.concat([x_m.iloc[l_m:, :], x_n.iloc[l_n:, :]])
y_train = pd.concat([y_m.iloc[:l_m], y_n.iloc[:l_n]])
y_test = pd.concat([y_m.iloc[l_m:], y_n.iloc[l_n:]])

y_test.reset_index(drop = True, inplace = True)

In [15]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, weights = 'distance')
classifier.fit(x_train, y_train)

y_pred = classifier.predict(x_test)

#part that shows which companies are "problematic"
# comps_m = manip_days.iloc[l_m:, 7]
# comps_n = manip_days.iloc[l_n:, 7] 
# comps = pd.concat([comps_m, comps_n])
# comps.reset_index(drop = True, inplace = True)
# for i in y_test.index:
#     #print(y_test.iloc[i])
#     #print(y_pred[i])
#     if y_test.iloc[i] == 1 and y_pred[i] == 0:
#         print(comps.iloc[i])

from sklearn.metrics import classification_report, confusion_matrix
print("KNN results:")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

KNN results:
[[292  12]
 [  7  86]]
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       304
           1       0.88      0.92      0.90        93

    accuracy                           0.95       397
   macro avg       0.93      0.94      0.93       397
weighted avg       0.95      0.95      0.95       397



In [16]:
#NB
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb = gnb.fit(x_train, y_train)
y_pred = gnb.predict(x_test)

print("NB reulst:")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

NB reulst:
[[290  14]
 [ 85   8]]
              precision    recall  f1-score   support

           0       0.77      0.95      0.85       304
           1       0.36      0.09      0.14        93

    accuracy                           0.75       397
   macro avg       0.57      0.52      0.50       397
weighted avg       0.68      0.75      0.69       397



In [17]:
#DT
from sklearn import tree

tr = tree.DecisionTreeClassifier()
tr = tr.fit(x_train, y_train)
y_pred = tr.predict(x_test)

print("DT results:")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

#tree.plot_tree(tr, max_depth = 1)

DT results:
[[290  14]
 [  5  88]]
              precision    recall  f1-score   support

           0       0.98      0.95      0.97       304
           1       0.86      0.95      0.90        93

    accuracy                           0.95       397
   macro avg       0.92      0.95      0.94       397
weighted avg       0.95      0.95      0.95       397



In [18]:
#ANN
from sklearn.neural_network import MLPClassifier
classifier = MLPClassifier(hidden_layer_sizes=(240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240), max_iter=20000000)
classifier = classifier.fit(x_train, y_train)

y_pred = classifier.predict(x_test)

print("ANN results:")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

ANN results:
[[295   9]
 [ 10  83]]
              precision    recall  f1-score   support

           0       0.97      0.97      0.97       304
           1       0.90      0.89      0.90        93

    accuracy                           0.95       397
   macro avg       0.93      0.93      0.93       397
weighted avg       0.95      0.95      0.95       397

